In [11]:
%load_ext autoreload
%autoreload 2
# !pip install ipywidgets
import sys
sys.path.append("/home/boittier/Documents/phd/ff_energy")

from ff_energy.structure import Structure
from ff_energy.job import Job
from ff_energy.jobmaker import get_structures_pdbs, JobMaker
from ff_energy.plot import plot_energy_MSE
from ff_energy.configmaker import *
from ff_energy.make_jobs_test import MakeJob

from pathlib import Path
import pandas as pd

# s = Structure("/home/boittier/charmm/mix3/jobs/pdbs/mix0.pdb")

atom_types = {
              # ("LIG", "O"): "OG311",
              # ("LIG", "C"): "CG331",
              # ("LIG", "H1"): "HGP1",
              # ("LIG", "H2"): "HGA3",
              # ("LIG", "H3"): "HGA3",
              # ("LIG", "H4"): "HGA3",
              ("TIP3", "OH2"): "OT",
              ("TIP3", "H1"): "HT",
              ("TIP3", "H2"): "HT",
              # ("LIG", "O"): "OT",
              # ("LIG", "H1"): "HT",
              # ("LIG", "H"): "HT",
              # ("LIG", "H2"): "HT",
              }

# def MakeJob(name, ConfigMaker, atom_types=atom_types):
#     structures, pdbs = get_structures_pdbs(
#         Path(ConfigMaker.pdbs),
#         atom_types=atom_types
#     ) 
#     return JobMaker(name, pdbs, structures, ConfigMaker.make().__dict__)






The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'ff_energy.make_jobs_test'

In [ ]:
structures, pdbs = get_structures_pdbs(
    Path(""),
    atom_types=atom_types
) 

In [2]:
CMS = []
for system in system_names:
    for theory in THEORY.keys():
        for elec in ["pc", "mdcm"]:
            print(system, theory, elec)
            cm = ConfigMaker(theory, system, elec)
            # print(cm.pdbs)
            print(cm.make().kwargs)
            print()
            CMS.append(cm)

water_cluster pbe0dz pc
<bound method Config.kwargs of m_basis: avdz
m_method: {ks,pbe0}
c_files: ['pbe0_dz.pc']
c_dcm_command: open unit 11 card read name p 
DCM IUDCM 11 TSHIFT XYZ 15
m_nproc: 8
m_memory: 150
m_queue: short
chmpath: /home/boittier/dev-release-dcm/build/cmake/charmm
modules: module load cmake/cmake-3.23.0-gcc-11.2.0-openmpi-4.1.3
>

water_cluster pbe0dz mdcm
<bound method Config.kwargs of m_basis: avdz
m_method: {ks,pbe0}
c_files: ['pbe0_dz.mdcm']
c_dcm_command: open unit 11 card read name p 
DCM IUDCM 11 TSHIFT XYZ 15
m_nproc: 8
m_memory: 150
m_queue: short
chmpath: /home/boittier/dev-release-dcm/build/cmake/charmm
modules: module load cmake/cmake-3.23.0-gcc-11.2.0-openmpi-4.1.3
>

water_dimer pbe0dz pc
<bound method Config.kwargs of m_basis: avdz
m_method: {ks,pbe0}
c_files: ['pbe0_dz.pc']
c_dcm_command: open unit 11 card read name p 
DCM IUDCM 11 TSHIFT XYZ 15
m_nproc: 8
m_memory: 150
m_queue: short
chmpath: /home/boittier/dev-release-dcm/build/cmake/charmm
modules

In [3]:
CMS[0].theory

{'m_basis': 'avdz', 'm_method': '{ks,pbe0}'}

In [10]:
jobmakers = []
for cms in CMS:
    jm = MakeJob(f"{cms.system_name}/{cms.theory_name}", cms, 
                 atom_types=cms.atom_types,system_name=cms.system_name)
    HOMEDIR = "/home/boittier/bhome"
    # jm.make_charmm(HOMEDIR)
    # jm.make_coloumb(HOMEDIR,f"/home/boittier/pcbach/{cms.system_type}/{cms.theory_name}/""{}/monomers")
    jobmakers.append(jm)

TypeError: MakeJob() got an unexpected keyword argument 'system_name'

In [ ]:
from ff_energy.slurm import SlurmJobHandler

shj = SlurmJobHandler(max_jobs=100)
print("Running jobs: ", shj.get_running_jobs())

for jm in jobmakers:
    for js in jm.get_cluster_jobs(HOMEDIR):
        shj.add_job(js)

print("Jobs: ", len(jm.get_cluster_jobs(HOMEDIR)))

shj.submit_jobs()